# Kahneman–Tversky Optimization (KTO) at scale with LoRA

This guide provides a step-by-step workflow for preference fine-tuning the [`meta-llama/Meta-Llama-3-8B-Instruct`](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct) model on a multi-GPU Anyscale cluster. You use **LLaMA-Factory** as the training framework and **LoRA** to reduce memory footprint and enable efficient multi-GPU training.

**KTO aligns a model to human preferences using **single binary labels (accept or reject)** instead of pairwise “chosen versus rejected” comparisons. KTO directly optimizes the policy on these unary signals, simplifying data preparation while still encouraging preferred behavior and discouraging undesired outputs.

## Step 1: Set up your environment

### Dependencies
First, ensure your environment has the correct libraries. Start with a pre-built container image and install LLaMA-Factory and DeepSpeed on top of it.

Recommended container image:
```bash
anyscale/ray-llm:2.48.0-py311-cu128
```

Execute the following commands to install the required packages and optional tools for experiment tracking and faster model downloads.

In [ ]:
%%bash
# Install the specific version of LLaMA-Factory
pip install -q llamafactory==0.9.3

# (Optional) For accelerated model downloads from Hugging Face
pip install -q hf_transfer==0.1.9

# (Optional) Acceleration methods (ensure CUDA/Torch compatibility)
pip install -q flash-attn==2.8.3 liger-kernel==0.6.2

# (Optional) Experiment tracking library
pip install -q mlflow==3.4.0

### Model and compute resources

| Item | Value |
|------|-------|
| **Base model** | [`meta-llama/Meta-Llama-3-8B-Instruct`](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct) |
| **Workers** | 4 × L40S / A100 (1 GPU each) |

Compared to SFT, KTO typically holds two copies of the model (policy and reference), and alignment datasets often use long contexts, so Anyscale recommends GPUs with larger VRAM. Techniques such as **LoRA** and memory-efficient attention can further reduce memory pressure.

## Step 2: Prepare the dataset

### Understand the dataset
This tutorial uses `kto_en_demo`, a unary-preference dataset for KTO. Each record contains a multi-turn ShareGPT-style dialogue with a **binary label** indicating whether the modeled behavior is preferred.

This dataset contains:
- `messages`: Turn-by-turn chat between a user and the assistant.
- `label`: A boolean (`true` or `false`) indicating whether the example is preferred.

**Note:** To maintain role alignment in ShareGPT format, you must follow a strict turn order: `human` and `observation` (tool output) must appear in odd-numbered positions, while `gpt` and `function_call` must appear in even-numbered positions. The model learns to generate the content in the `gpt` and `function_call` turns.

**Dataset example**
```json
{
"messages": [
    { "role": "user", "content": "Compare and contrast the roles of the hippocampus and the prefrontal cortex..." },
    { "role": "assistant", "content": "The human brain is a highly complex organ, responsible for a myriad of cognitive functions..." },
    { "role": "user", "content": "Discuss the mechanisms through which the prefrontal cortex ..." },
    { "role": "assistant", "content": "The prefrontal cortex (PFC)..." },
    { "role": "user", "content": "Can you elaborate on the role of the amygdala..." },
    { "role": "assistant", "content": "The amygdala plays a crucial role in the emotional processing of stored memories..." }
],
"label": true
}
```

### Register the dataset

To specify new datasets that are accessible across Ray worker nodes, you must first add a **`dataset_info.json`** to **[storage shared across nodes](https://docs.anyscale.com/configuration/storage#shared)** such as `/mnt/cluster_storage`. This configuration file acts as a central registry for all your datasets. It maps a custom name to your dataset file location, format, and column structure. 

If you plan to run KTO post-training on the `kto_en_demo` dataset, first complete the setup steps below. Ensure that you place the dataset files in a storage location that all workers can access (for example, a shared mount or object storage). Avoid storing large files on the head node. 

`dataset_info.json`

- `kto_tag` maps the unary preference label used by KTO.
- `tags` helps the loader interpret role/content fields in ShareGPT-style records.

```json
{
  "my_kto_en_demo": {
    "file_name": "/mnt/cluster_storage/kto_en_demo.json",
    "formatting": "sharegpt",
    "columns": {
      "messages": "messages",
      "kto_tag": "label"
    },
    "tags": {
      "role_tag": "role",
      "content_tag": "content",
      "user_tag": "user",
      "assistant_tag": "assistant"
    }
  }
}
```

For a more detailed dataset preparation and formatting guide, see [Choose your data format](https://docs.anyscale.com/llm/fine-tuning/data-preparation#kto).

In [ ]:
%%bash
# Make sure all files are accessible to worker nodes
# Create a copy of the data in /mnt/cluster_storage
wget https://anyscale-public-materials.s3.us-west-2.amazonaws.com/llm-finetuning/llama-factory/datasets/sharegpt/kto_en_demo.json -O /mnt/cluster_storage/kto_en_demo.json
# Create a copy of the dataset registry in /mnt/cluster_storage
cp ../dataset-configs/dataset_info.json /mnt/cluster_storage/

## Step 3: Create the preference-tuning config (KTO and LoRA)

Create a YAML file that defines your **KTO** run. It specifies the base model, dataset, **LoRA** settings, KTO hyperparameters, optional acceleration methods, logging, and Ray cluster resources.

**Important notes:**
- **Acceleration libraries:** You can use `flash_attn` and `liger-kernel` together, but actual speed and memory gains vary with GPU architecture, sequence length, batch size, precision, kernel availability. Benchmark your training workloads to confirm improvements. Note that `fa2` isn't supported on Turing GPUs (e.g., T4).
- **Access and paths:** The YAML only needs to be on the **head node**, but any referenced paths (for example, `dataset_dir`, `ray_storage_path`, `output_dir`) must be on **shared storage** (such as `/mnt/cluster_storage/`) visible to all workers.
- **Gated models:** If your base model has gated access on Hugging Face, set `HF_TOKEN` in the runtime environment.
- **Memory tips:** If VRAM is tight, consider switching to [QLoRA]((https://github.com/ray-project/ray/blob/master/doc/source/ray-overview/examples/llamafactory-llm-fine-tune/notebooks/dpo_qlora.ipynb)) (4/8-bit) and adding the corresponding quantization keys.

### Configure LLaMA-Factory with Ray

**Note**: To customize the training configuration, edit `train-configs/kto_lora.yaml`. 

```yaml
# kto_lora.yaml

### model
model_name_or_path: meta-llama/Meta-Llama-3-8B-Instruct
trust_remote_code: true

### method
stage: kto
do_train: true
finetuning_type: lora
lora_rank: 8
lora_target: all
pref_beta: 0.1

### acceleration methods
# You can enable both methods at the same time
flash_attn: fa2            # Speed up attention and cut activation memory at long context. Use auto on Turing GPUs (e.g., T4)
enable_liger_kernel: true  # Reduce VRAM and improve throughput across multiple transformer ops

### dataset
dataset: my_kto_en_demo
dataset_dir: /mnt/cluster_storage

template: llama3
cutoff_len: 1024
max_samples: 1000
overwrite_cache: true
preprocessing_num_workers: 16

### output
output_dir: llama3_8b_lora_kto
logging_steps: 5
save_steps: 50
plot_loss: true
overwrite_output_dir: true
report_to: mlflow   # or none

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 2
num_train_epochs: 3.0  # Low for demo purpose; adjust as needed
learning_rate: 5.0e-6
bf16: true
lr_scheduler_type: cosine
warmup_ratio: 0.1
ddp_timeout: 180000000

### ray
ray_run_name: llama3_8b_kto_lora
ray_storage_path: /mnt/cluster_storage/
ray_num_workers: 4
resources_per_worker:
  GPU: 1
  anyscale/accelerator_shape:4xL40S: 0.001  # Pin a specific node shape
  # accelerator_type:L40S: 0.001            # or just request a GPU type

ray_init_kwargs:
  runtime_env:
    env_vars:
      # If using gated models like meta-llama/Llama-3-8B-Instruct
      HF_TOKEN: <your_huggingface_token>
      # Enable faster downloads if hf_transfer is installed:
      HF_HUB_ENABLE_HF_TRANSFER: '1'
      # If using mlflow for experiments tracking
      MLFLOW_TRACKING_URI: "https://<your_cloud_id>.cloud.databricks.com"
      MLFLOW_TRACKING_TOKEN: "<mlflow_tracking_token>"
      MLFLOW_EXPERIMENT_NAME: "/Users/<your_user_id>/experiment_name"
```

## Step 4: Train and monitor

**Note**: For gated models such as [`meta-llama/Meta-Llama-3-8B-Instruct`](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct), ensure that you accept the license agreement for the models on the Hugging Face site and set `HF_TOKEN` in the runtime environment. If you installed MLflow, configure its credentials. Otherwise, set `report_to: none` in `kto_lora.yaml` to avoid `api_token not set` errors.

With all configurations in place, you can launch fine-tuning or post-training in one of two ways:

### Option A: Run from a workspace (quick start)

The `USE_RAY=1` prefix tells LLaMA-Factory to run in distributed mode on the Ray cluster attached to your workspace.

In [ ]:
%%bash
USE_RAY=1 llamafactory-cli train ../train-configs/kto_lora.yaml

### Option B — Run as an Anyscale job (production)

For longer or production runs, submit the training as an **Anyscale job**. Jobs run outside your interactive session for better stability, retries, and durable logs. You package LLaMA-Factory and other libraries in a container image and launch with a short job config. See [Run LLaMA-Factory as an Anyscale job](https://docs.anyscale.com/llm/fine-tuning/llamafactory-jobs) for the step-by-step guide.

### Tracking with MLflow

If you enabled MLflow logging (`report_to: mlflow` in your YAML), LLaMA-Factory logs metrics (loss, learning rate, etc.), parameters, and artifacts to your configured MLflow tracking server.

**Example YAML snippet:**

```yaml
report_to: mlflow

ray_init_kwargs:
  runtime_env:
    env_vars:
      MLFLOW_TRACKING_URI: "https://<your_cloud_id>.cloud.databricks.com"
      MLFLOW_TRACKING_TOKEN: "<mlflow_tracking_token>"
      MLFLOW_EXPERIMENT_NAME: "/Users/<your_user_id>/experiment_name"
```

**MLFlow example**

![MLflow](https://anyscale-public-materials.s3.us-west-2.amazonaws.com/llm-finetuning/llama-factory/3.2.3/3.2.3-mlflow.png)

For a more detailed guide on tracking experiments with other tools such as Weights and Biases or MLflow, see [Observability and tracking](https://docs.anyscale.com/llm/fine-tuning/observability-and-tracking).

## Step 5: Locate checkpoints

Ray Train writes checkpoints under `ray_storage_path/ray_run_name`. In this example run, the path is: `/mnt/cluster_storage/llama3_8b_kto_lora`. 

Inside, you see a **trainer session** directory named like:
`TorchTrainer_75e12_00000_0_2025-09-22_17-58-47`.

- Ray Train creates `TorchTrainer_*` **when the trainer starts**; the suffix encodes a short run ID and the **start timestamp**.
- Within that directory, Ray Train names checkpoints `checkpoint_000xxx/`, where the number is the saved ordered checkpoints.

Control the save cadence with `save_strategy` and `save_steps`. For instructions on how to resume interrupted training with `resume_from_checkpoint` and more, see [Understand the artifacts directory](https://docs.anyscale.com/llm/fine-tuning/checkpointing#artifacts-directory).

## Step 6: Export the model

If you use LoRA, you can keep the base model and adapters separate for [multi-LoRA deployment](https://docs.anyscale.com/llm/serving/multi-lora) or [merge the adapters](https://docs.anyscale.com/llm/fine-tuning/checkpointing#merge-lora) into the base model for low-latency inference. 

For full fine-tuning or freeze-tuning, export the fine-tuned model directly.

You may optionally apply [post-training quantization](https://docs.anyscale.com/llm/fine-tuning/checkpointing#ptq) on merged or full models before serving.